In [113]:
import re
import os
import pyodbc

import pandas as pd

In [22]:
main_directory = '//enthhs0112.state.mt.ads/MIDIS/Rhapsody Msg Failed/2023'

In [69]:
# Define function to extract hte lab name
def extract_lab_name(value):
    if value is not None:
        return value.split('|')[3] if len(value.split('|')) > 3 else None
    else:
        return None

In [112]:
result_df_list = []  # List to store individual DataFrames

# Walk through each file in the directory
for root, dirs, files in os.walk(main_directory):
    for file_name in files:
        file_path = os.path.join(root, file_name)

        result_list = []  # List to store data from each file

        with open(file_path, 'r') as file:
            data = file.read()
            sublists = data.split("_____________________________________________________________________________________________________________")

            for sublist in sublists:
                inner_list = sublist.strip().split('\n')
                result_list.append(inner_list)

        error_messages = []

        for sublist in result_list:
            if len(sublist) > 1:
                # Split the first element into "date" and "filename"
                date_filename_parts = sublist[0].split("Input Filename: ")
                if len(date_filename_parts) == 2:
                    date = date_filename_parts[0].strip()
                    filename = date_filename_parts[1].strip()
                else:
                    date = "N/A"
                    filename = "N/A"

                # Capture everything between 'Error: ' and 'Failed Message: '
                sender = sublist[3].split("Sender: ")[-1].strip() if len(sublist) > 3 else None
                error_route = sublist[5].split("Error Route: ")[-1].strip() if len(sublist) > 3 else None
                error_filter = sublist[6].split("Error Filter: ")[-1].strip() if len(sublist) > 3 else None

                # Get the MSH of the failed message
                failed_message_index = sublist.index('  Failed Message:  ')  # Find the index of '  Failed Message:  '
                failed_msg_MSH = sublist[failed_message_index + 1] if failed_message_index < len(sublist) - 1 else None

                # Get the MSH of the original message
                orig_message_index = sublist.index('  Original Message:  ') if '  Original Message:  ' in sublist else None  
                orig_msg_MSH = sublist[orig_message_index + 1] if orig_message_index is not None and orig_message_index < len(sublist) - 1 else None


                concatenated_list = [date, 
                                    sender, 
                                    filename,
                                    error_route, 
                                    error_filter, 
                                    failed_msg_MSH, 
                                    orig_msg_MSH
                                    ]
                error_messages.append(concatenated_list)



        # Define column names
        columns = ['date', 
                   'sender', 
                   'filename', 
                   'error_route', 
                   'error_filter', 
                   'failed_msg_MSH', 
                   'facility'
                   ]

        # Create DataFrame for each file
        df = pd.DataFrame(error_messages, columns=columns)

        # Apply the lab name extract function to the DataFrame column
        if orig_message_index is not None:
            df['facility'] = df['facility'].apply(extract_lab_name)

        # Append the DataFrame to the result_df_list
        result_df_list.append(df)

# Concatenate all DataFrames in result_df_list into a final DataFrame
result_df = pd.concat(result_df_list, ignore_index=True)

result_df['date'] = result_df['date'].astype(str).str[:10]

# Split the facility column into facility, CLIA number, and CLIA
result_split = result_df['facility'].str.split('^', expand=True)

# Renaming the columns with meaningful names
result_split.columns = [f'facility.{i+1}' for i in range(result_split.shape[1])]

# Concatenating the new columns with the original DataFrame
result_df = pd.concat([result_df, result_split], axis=1)

# Dropping the original column that was split
cols_to_drop = ['facility', 'facility.3']
result_df = result_df.drop(cols_to_drop, axis=1)

result_df = result_df.rename(columns={'facility.1':'facility', 
                                'facility.2':'CLIA'
})


# Fill None values in the sender column
mask = result_df['sender'] == "null"
result_df.loc[mask, 'sender'] = result_df.loc[mask, 'facility']

result_df['facility'].fillna(result_df['sender'], inplace=True)

result_df['countervar'] = 1

# Display the final DataFrame
result_df.head(5)


,date,sender,filename,error_route,error_filter,failed_msg_MSH,facility,CLIA,countervar
0,2023-04-01,Bozeman Health Deaconess Hospital,ELR_bhdh_1079284_20230401105132936.txt,(1418) 4 b - ELR MT 2,(1390) InsertTranslationActivity SITSD,MSH|^~\&#|EPIC||||20230401105130000-0600|LABBA...,,None,1
1,2023-04-02,St Peters Hospital,ELR_sph_1079659_610335777.XO,(592) 3 - Map to 2.5.1 and Validate,(512) EDI Message Validation,MSH|^~\&#|MEDITECH LAB^1.3.6.1.4.1.24310^ISO|S...,STP,1.3.6.1.4.1.24310,1
2,2023-04-03,LABCORP,PHN_LabCorp_Labcorp_Fail.HL7,(1418) 4 b - ELR MT 2,(1401) SetTranslationActivity Variables,MSH|^~\&|LABCORP-CORP|LABCORP^34D0655059^CLIA|...,LABCORP,34D0655059,1
3,2023-04-03,LABCORP,PHN_LabCorp_Labcorp_Fail.HL7,(1418) 4 b - ELR MT 2,(1401) SetTranslationActivity Variables,MSH|^~\&|LABCORP-CORP|LABCORP^34D0655059^CLIA|...,LABCORP,34D0655059,1
4,2023-04-05,Cerner,null,(592) 3 - Map to 2.5.1 and Validate,(512) EDI Message Validation,MSH|^~\&|HealthSentry^2.16.840.1.113883.3.13.2...,BillingsClinicHosp,27D0409764,1


In [140]:
# Connection parameters
database_path = r'//state.mt.ads/HHS/Shared/PHSD/DIV-SHARE/OESS/Surveillance and Informatics Section/Special Projects/ELR_data_quality_monitoring/elr-data-monitoring-system/ELRDQMS.accdb'

# Connection string for Access database
connection_string = f'DRIVER={{Microsoft Access Driver (*.mdb, *.accdb)}};DBQ={database_path};'

# Establish a connection to the database
connection = pyodbc.connect(connection_string)

# Create a cursor to execute SQL queries
cursor = connection.cursor()


In [ ]:

# Set your ODBC connection string
conn_str = (
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
    f'DBQ={database_path};'
)

# Establish a connection to the Access database
conn = pyodbc.connect(conn_str)

# Define the name of the table in the Access database
table_name = 'error_msgs'

# Create a cursor for executing SQL commands
cursor = conn.cursor()

# Test it out
sql="Insert into error_msgs (msg_date, sender, filename, error_route, error_filter, failed_msg_MSH, facility, CLIA, countervar) values (1, 'Peter','Jackson', 'w', 'w', 't', 't', 'y', 't')"

cursor.execute(sql)

# Insert each row from the DataFrame into the Access table
# for index, row in result_df.iterrows():
#     insert_query = f"INSERT INTO {table_name} (msg_date, sender, filename, error_route, error_filter, failed_msg_MSH, facility, CLIA, countervar) VALUES (?, ?, ?,?,?,?,?,?,?)"
#     cursor.execute(insert_query, row['msg_date'], row['sender'], row['filename'], row['error_route'], row['error_filter'], row['failed_msg_MSH'], row['facility'], row['CLIA'], row['countervar'])

# Commit the changes
conn.commit()

# Close the cursor and the database connection
cursor.close()
conn.close()


DataError: ('22018', '[22018] [Microsoft][ODBC Microsoft Access Driver] Data type mismatch in criteria expression. (-3030) (SQLExecDirectW)')